In [1]:
%pip uninstall -y torch torchvision torchaudio 

%pip install --no-cache-dir \
  torch torchvision torchaudio \
  --index-url https://download.pytorch.org/whl/cpu

%pip uninstall -y transformers datasets

%pip install --no-cache-dir \
  transformers datasets

%pip install transformers[torch]

%pip install 'accelerate>=0.26.0'

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0
Found existing installation: torchvision 0.22.0
Uninstalling torchvision-0.22.0:
  Successfully uninstalled torchvision-0.22.0
Found existing installation: torchaudio 2.7.0
Uninstalling torchaudio-2.7.0:
  Successfully uninstalled torchaudio-2.7.0
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio]3 [torchaudio]]
Note: you may need to restart the kernel to use updated packages.
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-

In [2]:
import os

folder = "outputs/bigcode-ts-output-50000"
file_count = sum(len(files) for _, _, files in os.walk(folder))
print("Total files:", file_count)

Total files: 46056


In [6]:
# ─── 1) Imports ─────────────────────────────────────────────────────────────
import glob, os, math, torch, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments
)

# ─── 2) Configuration ───────────────────────────────────────────────────────
DATA_DIR     = "outputs/bigcode-ts-output-4000"  # folder with .ts/.tsx files
METADATA_CSV = "outputs/ts-output-4000-types.csv"                     # CSV with file, interfaces, types, enums, etc.
BATCH_SIZE   = 4
MAX_LENGTH   = 512
EPOCHS       = 3
OUTPUT_DIR   = "outputs/typescriptmate-model"

# ─── 3) Load metadata and build initial Dataset ────────────────────────────
df = pd.read_csv(METADATA_CSV)
# Ensure file paths are correct: you may need to prepend DATA_DIR
# If 'File' column is relative, adjust accordingly:
# df['file'] = df['File'].apply(lambda p: os.path.join(DATA_DIR, p))

dataset_meta = Dataset.from_pandas(df, preserve_index=False)

# Read file contents into 'text'
def add_text(example):
    path = example['file']
    try:
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            example['text'] = f.read()
    except FileNotFoundError:
        example['text'] = ''
    return example

dataset_all = dataset_meta.map(add_text, batched=False)

# ─── 4) Split into train / validation ───────────────────────────────────────
splits = dataset_all.train_test_split(test_size=0.1, seed=42)
datasets = DatasetDict(train=splits['train'], validation=splits['test'])

# ─── 5) Type-Aware Annotation using CSV metadata ────────────────────────────
# Combine all declared and used types into one list per example
TYPE_COLUMNS = ['interfaces', 'types', 'enums', 'classes', 'decorators', 'imports', 'exports', 'usedTypes']

def annotate_metadata(example):
    all_types = []
    for col in TYPE_COLUMNS:
        val = example.get(col) or []
        if isinstance(val, str):
            # assume comma-separated in CSV
            val = [v.strip() for v in val.split(',') if v.strip()]
        all_types.extend(val)
    # dedupe
    example['all_types'] = list(dict.fromkeys(all_types))
    return example

datasets = DatasetDict({
    split: ds.map(annotate_metadata, batched=False)
    for split, ds in datasets.items()
})

# Filter to examples with at least one type annotation
datasets = DatasetDict({
    split: ds.filter(lambda ex: len(ex['all_types']) > 0)
    for split, ds in datasets.items()
})

print("After metadata filtering:")
print("  • train:", len(datasets['train']), "examples")
print("  • valid:", len(datasets['validation']), "examples")

# ─── 6) Tokenization ────────────────────────────────────────────────────────
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(
        batch['text'],
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH
    )

tokenized = datasets.map(
    tokenize_fn,
    batched=True,
    remove_columns=[*TYPE_COLUMNS, 'file', 'text']
)

# ─── 7) Data collator & model ──────────────────────────────────────────────
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
model = AutoModelForCausalLM.from_pretrained("gpt2")

# ─── 8) Training arguments ─────────────────────────────────────────────────
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    save_steps=500,
    logging_steps=200,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    do_train=True,
    do_eval=True
)

# ─── 9) Trainer initialization & train ──────────────────────────────────────
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    data_collator=data_collator
)

trainer.train()

# ─── 10) Evaluation & save ──────────────────────────────────────────────────
eval_results = trainer.evaluate()
print("Full eval results:", eval_results)
if 'eval_loss' in eval_results:
    print("Perplexity:", math.exp(eval_results['eval_loss']))

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

Found 46055 TypeScript files on disk


Generating train split: 46055 examples [00:08, 5319.89 examples/s]
Map: 100%|██████████| 4606/4606 [00:03<00:00, 1262.54 examples/s]


Step,Training Loss
200,1.889500
400,1.660700
600,1.585100
800,1.536700
1000,1.526900
1200,1.470300
1400,1.447200
1600,1.426400
1800,1.414300
2000,1.398300


/opt/homebrew/Caskroom/miniconda/base/envs/dissertation/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniconda/base/envs/dissertation/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniconda/base/envs/dissertation/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Full eval results: {'eval_loss': 1.0589054822921753, 'eval_runtime': 347.146, 'eval_samples_per_second': 13.268, 'eval_steps_per_second': 3.318, 'epoch': 3.0}
Validation Perplexity: 2.88


('outputs/typescriptmate-model/tokenizer_config.json',
 'outputs/typescriptmate-model/special_tokens_map.json',
 'outputs/typescriptmate-model/vocab.json',
 'outputs/typescriptmate-model/merges.txt',
 'outputs/typescriptmate-model/added_tokens.json',
 'outputs/typescriptmate-model/tokenizer.json')

In [7]:
print("Full eval results:", eval_results)

if "eval_loss" in eval_results:
    ppl = math.exp(eval_results["eval_loss"])
    print(f"Validation Perplexity: {ppl:.2f}")
else:
    print("No 'eval_loss' in eval_results; keys are:", list(eval_results.keys()))

Full eval results: {'eval_loss': 1.0589054822921753, 'eval_runtime': 347.146, 'eval_samples_per_second': 13.268, 'eval_steps_per_second': 3.318, 'epoch': 3.0}
Validation Perplexity: 2.88
